## CNN for music genre classification

In [102]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.initializers import HeNormal


In [61]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available: 1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Folder setup

In [62]:
SPEC_TRAIN_FOLDER = "Data/Spectograms/train"
SPEC_TEST_FOLDER = "Data/Spectograms/test"
SPEC_VAL_FOLDER = "Data/Spectograms/val"

## Constants

In [63]:
IMAGE_SIZE = (130, 128)

In [64]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=SPEC_TRAIN_FOLDER,
    labels='inferred',
    label_mode='int',
    image_size=IMAGE_SIZE,
    batch_size=32,
)

Found 5995 files belonging to 10 classes.


In [65]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=SPEC_TEST_FOLDER,
    labels='inferred',
    label_mode='int',
    image_size=IMAGE_SIZE,
    batch_size=32,
    color_mode='rgb'
)

Found 1998 files belonging to 10 classes.


In [66]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=SPEC_VAL_FOLDER,
    labels='inferred',
    label_mode='int',
    image_size=IMAGE_SIZE,
    batch_size=32,
    color_mode='rgb'
)

Found 1998 files belonging to 10 classes.


In [ ]:
def get_model(input_shape):
    input_shape = (input_shape[0], input_shape[1], 3)
    model = Sequential([
        Conv2D(32, kernel_size=3, padding='same', activation='relu', kernel_initializer=HeNormal(), input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(64, kernel_size=3, padding='same', activation='relu', kernel_initializer=HeNormal()),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(128, kernel_size=3, padding='same', activation='relu', kernel_initializer=HeNormal()),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        Dropout(0.3),

        Flatten(),
        Dense(128, activation='relu', kernel_initializer=HeNormal()),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model


In [83]:
def compile_model(model):
   model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [70]:
def train_model(model, train_ds, epochs):
    model.fit(train_ds, epochs=epochs)

In [94]:
model = get_model(IMAGE_SIZE)
model.summary()

/home/diego/GenreGenie/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 130, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 65, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 65, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 65, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,289,866 (16.36 MB)

 Trainable params: 4,289,418 (16.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [95]:
compile_model(model)

In [96]:
train_model(model, train_ds, 30)

Epoch 1/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.1893 - loss: 3.6055
Epoch 2/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.1650 - loss: 2.1994
Epoch 3/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.1921 - loss: 2.1068
Epoch 4/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.2147 - loss: 2.0179
Epoch 5/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2428 - loss: 1.9779
Epoch 6/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2564 - loss: 1.9328
Epoch 7/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2644 - loss: 1.8563
Epoch 8/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2776 - loss: 1.7827
Epoch 9/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.3078 - loss: 1.7307
Epoch 10/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.3037 - loss: 1.7572
Epoch 11/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3258 - loss: 1.7501 
Epoch 12/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 

In [100]:
test_loss, test_acc = model.evaluate(test_ds)

print('\nTest accuracy:', test_acc)
print('\n Test lost:', test_loss)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6703 - loss: 1.1952

Test accuracy: 0.684684693813324

 Test lost: 1.2239911556243896


In [101]:
test_loss, test_acc = model.evaluate(val_ds)
print('\nValidation accuracy:', test_acc)
print('\n Validation lost:', test_loss)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6509 - loss: 1.3825

Validation accuracy: 0.6401401162147522

 Validation lost: 1.4030405282974243
